<a href="https://colab.research.google.com/github/brendanpshea/database_sql/blob/main/data/MarioBrosPlumbing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Connect sql magic
%load_ext sql
%sql sqlite:///mario_bros_plumbing.db

In [2]:
%%sql
CREATE TABLE Customers (
  customer_id INTEGER PRIMARY KEY,
  first_name TEXT,
  last_name TEXT,
  address TEXT,
  phone_number TEXT
);

CREATE TABLE Employees (
  employee_id INTEGER PRIMARY KEY,
  first_name TEXT,
  last_name TEXT,
  job_title TEXT,
  hire_date TEXT
);

CREATE TABLE Services (
  service_id INTEGER PRIMARY KEY,
  service_name TEXT,
  description TEXT,
  price REAL
);

CREATE TABLE Orders (
  order_id INTEGER PRIMARY KEY,
  customer_id INTEGER,
  employee_id INTEGER,
  order_date TEXT,
  total_amount REAL,
  FOREIGN KEY (customer_id) REFERENCES Customers (customer_id),
  FOREIGN KEY (employee_id) REFERENCES Employees (employee_id)
);

CREATE TABLE Order_Items (
  order_item_id INTEGER PRIMARY KEY,
  order_id INTEGER,
  service_id INTEGER,
  quantity INTEGER,
  FOREIGN KEY (order_id) REFERENCES Orders (order_id),
  FOREIGN KEY (service_id) REFERENCES Services (service_id)
);

 * sqlite:///mario_bros_plumbing.db
Done.
Done.
Done.
Done.
Done.


[]

In [3]:
%%sql
INSERT INTO Employees (employee_id, first_name, last_name, job_title, hire_date)
VALUES
  (1, 'Mario', 'Mario', 'Master Plumber', '1985-09-13'),
  (2, 'Luigi', 'Mario', 'Plumber', '1986-02-20'),
  (3, 'Toad', 'Toadstool', 'Apprentice Plumber', '2020-11-05');

 * sqlite:///mario_bros_plumbing.db
3 rows affected.


[]

In [4]:
%%sql
INSERT INTO Customers (customer_id, first_name, last_name, address, phone_number)
VALUES
  (1, 'Princess', 'Peach', 'Mushroom Castle, Toad Town', '(555) 123-4567'),
  (2, 'Yoshi', 'Dino', '24 Egg Island, Dinosaur Land', '(555) 987-6543'),
  (3, 'Daisy', 'Sarasa', '10 Sarasaland Way, Chai Kingdom', '(555) 456-7890'),
  (4, 'Toadette', 'Toadstool', '15 Mushroom St, Toad Town', '(555) 789-0123'),
  (5, 'Bowser', 'Koopa', '1 Bowser Castle, Dark Land', '(555) 654-3210'),
  (6, 'Wario', 'Wario', '100 Gold Coin Blvd, Diamond City', '(555) 321-0987'),
  (7, 'Waluigi', 'Wario', '101 Silver Coin Ave, Diamond City', '(555) 098-7654'),
  (8, 'Donkey', 'Kong', '50 Banana Jungle, DK Island', '(555) 111-2222'),
  (9, 'Diddy', 'Kong', '51 Banana Jungle, DK Island', '(555) 333-4444'),
  (10, 'Rosalina', 'Cosmic', 'Comet Observatory, Space', '(555) 555-6666'),
  (11, 'Cappy', 'Bonneter', '1 Hat Kingdom Way, Cap Kingdom', '(555) 777-8888'),
  (12, 'Pauline', 'Toyopolis', '1 New Donk City Plaza, Metro Kingdom', '(555) 999-0000');

 * sqlite:///mario_bros_plumbing.db
12 rows affected.


[]

In [5]:
%%sql
INSERT INTO Services (service_id, service_name, description, price)
VALUES
  (1, 'Pipe Repair', 'Fix leaky or broken pipes', 50.00),
  (2, 'Drain Cleaning', 'Clear clogged drains and pipes', 75.00),
  (3, 'Toilet Installation', 'Install a new toilet', 150.00),
  (4, 'Sink Replacement', 'Replace an old or damaged sink', 200.00),
  (5, 'Water Heater Repair', 'Fix issues with water heaters', 120.00),
  (6, 'Plumbing Inspection', 'Inspect plumbing systems for potential issues', 100.00),
  (7, 'Emergency Service', '24/7 emergency plumbing service', 250.00);

 * sqlite:///mario_bros_plumbing.db
7 rows affected.


[]

In [9]:
import sqlite3
import random
from datetime import datetime, timedelta

# Connect to the SQLite database
conn = sqlite3.connect('mario_bros_plumbing.db')
cursor = conn.cursor()

# Generate 500 random orders
for _ in range(500):
    # Select a random customer with weighted probabilities
    cursor.execute("SELECT customer_id FROM Customers ORDER BY RANDOM() * CASE "
                   "WHEN customer_id IN (1, 2, 3) THEN 3 "  # Customers 1, 2, and 3 are 3 times more likely
                   "WHEN customer_id IN (4, 5, 6) THEN 2 "  # Customers 4, 5, and 6 are 2 times more likely
                   "ELSE 1 END LIMIT 1")
    result = cursor.fetchone()
    if result is None:
        continue  # Skip this iteration if no customer is found
    customer_id = result[0]

    # Generate a random order date between 1985 and 2024
    start_date = datetime(1985, 1, 1)
    end_date = datetime(2024, 12, 31)
    time_between_dates = end_date - start_date
    days_between_dates = time_between_dates.days
    random_number_of_days = random.randrange(days_between_dates)
    order_date = start_date + timedelta(days=random_number_of_days)

    # Select a random employee hired before the order date
    employee_id = None  # Initialize employee_id with a default value
    cursor.execute("SELECT employee_id FROM Employees WHERE hire_date <= ? ORDER BY RANDOM() LIMIT 1",
                   (order_date.strftime('%Y-%m-%d'),))
    result = cursor.fetchone()
    if result is not None:
        employee_id = result[0]

    if employee_id is None:
        continue  # Skip this iteration if no suitable employee is found

    # Insert the order into the Orders table
    cursor.execute("INSERT INTO Orders (customer_id, employee_id, order_date) VALUES (?, ?, ?)",
                   (customer_id, employee_id, order_date.strftime('%Y-%m-%d')))
    order_id = cursor.lastrowid

    # Generate random order items for the order
    num_items = random.randint(1, 5)
    for _ in range(num_items):
        # Select a random service with weighted probabilities
        cursor.execute("SELECT service_id, price FROM Services ORDER BY RANDOM() * CASE "
                       "WHEN service_id IN (1, 2) THEN 3 "  # Services 1 and 2 are 3 times more likely
                       "WHEN service_id IN (3, 4) THEN 2 "  # Services 3 and 4 are 2 times more likely
                       "ELSE 1 END LIMIT 1")
        result = cursor.fetchone()
        if result is None:
            continue  # Skip this iteration if no service is found
        service_id, price = result

        # Generate a random quantity
        quantity = random.randint(1, 3)

        # Insert the order item into the Order_Items table
        cursor.execute("INSERT INTO Order_Items (order_id, service_id, quantity) VALUES (?, ?, ?)",
                       (order_id, service_id, quantity))

    # Update the total amount for the order
    cursor.execute("UPDATE Orders SET total_amount = (SELECT SUM(Services.price * Order_Items.quantity) "
                   "FROM Order_Items JOIN Services ON Order_Items.service_id = Services.service_id "
                   "WHERE Order_Items.order_id = ?) WHERE order_id = ?", (order_id, order_id))

# Commit the changes and close the connection
conn.commit()
conn.close()

In [10]:
%%sql
SELECT * FROM Order_Items;

 * sqlite:///mario_bros_plumbing.db
Done.


order_item_id,order_id,service_id,quantity
1,1,3,2
2,1,7,1
3,1,2,3
4,1,4,1
5,1,4,1
6,2,2,1
7,3,2,3
8,3,6,1
9,3,4,3
10,3,4,3
